### <font color='blue'>Import all packages</font> ###

In [ ]:
from google.colab import drive
#drive.flush_and_unmount()
drive.mount('/content/drive')
#drive.mount('/content/drive', force_remount=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
!pip install transformers
!pip install sentencepiece
!pip install tensorflow==2.7.0
!pip install stanza
!pip install transformers
!pip install tensorflow-addons
!pip install nltk
!pip install textacy

In [1]:

import tensorflow as tf
print(tf.__version__)

2.7.0


In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn.model_selection import train_test_split
from textacy.datasets.supreme_court import SupremeCourt
import numpy as np
import re
import unicodedata
import statistics
import nltk
#from transformers import pipeline
from nltk.corpus import stopwords
from tensorflow import keras
from tensorflow.keras.layers import Dense,Dropout, Input, BatchNormalization, Concatenate, Flatten
from tqdm import tqdm
import pickle
from sklearn.metrics import confusion_matrix,f1_score,classification_report
import matplotlib.pyplot as plt
import itertools
from sklearn.utils import shuffle
from tensorflow.keras import regularizers
#from transformers import *
from transformers import BertTokenizer, TFBertModel, BertConfig,TFDistilBertModel,DistilBertTokenizer,DistilBertConfig
import pandas as pd
from transformers import AutoTokenizer, TFAutoModel
import numpy as np
import gc
import math
import json
import stanza
from tensorflow.keras import *
import tensorflow as tf
from tensorflow.keras import *
import tensorflow.keras.backend as K
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from transformers import TFRobertaModel,RobertaTokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.initializers import RandomUniform

from numpy.random import seed
import random as python_random
import os
import sys

np.random.seed(1)
python_random.seed(1)
tf.random.set_seed(1)

/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-06-05 14:10:33.282947: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-05 14:10:34.917585: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78949 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:4b:00.0, compute capability: 8.0


### <font color='blue'> Preprocessing and cleaning functions </font> ###

In [ ]:
!cp "/content/drive/My Drive/labels_sc.txt" "./labels_sc.txt"
!cp "/content/drive/My Drive/labels_sc_279.txt" "./labels_sc_279.txt"

In [3]:
sc = SupremeCourt()
print(sc.info)
sc.download()

texts_complete=[]
texts_0_512 = []  # list of text samples
texts_512_1024 = []  # list of text samples
texts_1024_1536 = []  # list of text samples
texts_1536_2048 = []  # list of text samples
texts_2048_2560 = []  # list of text samples
texts_2560_3072 = []  # list of text samples
labels_index = {}  # dictionary mapping label name to numeric id
labels = []  # list of label ids

issue_codes = list(sc.issue_area_codes.keys()) # 15 labels
print(issue_codes)
issue_codes.sort()
issue_codes = [str(ic) for ic in issue_codes]

labels_index = dict(zip(issue_codes, np.arange(len(issue_codes))))
print(labels_index)
count=0

for record in sc.records():
        
        #print(record)
        count=count+1
        #print(count)
        if record[1]['issue'] == None: # some cases have None as an issue
            labels.append(labels_index['-1'])
        else:
            labels.append(labels_index[record[1]['issue'][:-4]])
        
        temp=record[0].split("Footnotes")[0]
        sentence=[]
        for word in temp.split():
            if any(ch.isdigit() for ch in word):
                continue
            else:
                sentence.append(word)
        new_sen=' '.join(sentence)
        new_new_sen=new_sen.split()
        first_512=new_new_sen[0:512]
        first_512_1024=new_new_sen[512:1024]
        first_512=' '.join(first_512)
        first_512_1024=' '.join(first_512_1024)

        first_1024_1536=new_new_sen[1024:1536]
        first_1536_2048=new_new_sen[1536:2048]
        first_1024_1536=' '.join(first_1024_1536)
        first_1536_2048=' '.join(first_1536_2048)

        first_2048_2560=new_new_sen[2048:2560]
        first_2560_3072=new_new_sen[2560:3072]
        first_2048_2560=' '.join(first_2048_2560)
        first_2560_3072=' '.join(first_2560_3072)
        
        texts_complete.append(new_sen)
        texts_0_512.append(first_512)
        texts_512_1024.append(first_512_1024)
        texts_1024_1536.append(first_1024_1536) 
        texts_1536_2048.append(first_1536_2048) 
        texts_2048_2560.append(first_2048_2560)  
        texts_2560_3072.append(first_2560_3072) 

len_list_complete = [len(ele.split()) for ele in texts_complete]
len_list_0_512 = [len(ele.split()) for ele in texts_0_512]
len_list_512_1024 = [len(ele.split()) for ele in texts_512_1024]
len_list_1024_1536 = [len(ele.split()) for ele in texts_1024_1536]
len_list_1536_2048 = [len(ele.split()) for ele in texts_1536_2048]
len_list_2048_2560 = [len(ele.split()) for ele in texts_2048_2560]
len_list_2560_3072 = [len(ele.split()) for ele in texts_2560_3072]

print(labels)
print(len(labels))


res_complete = 0 if len(len_list_complete) == 0 else (float(sum(len_list_complete)) / len(len_list_complete))
print("Average Length of complete text %s" % res_complete) 
print('Found %s texts in complete' % len(texts_complete))
print("Median Length: %s " % statistics.median(len_list_complete))
print("Max length of complete text %s" % max(len_list_complete))
print("Min length of complete text %s" % min(len_list_complete))


res_0_512 = 0 if len(len_list_0_512) == 0 else (float(sum(len_list_0_512)) / len(len_list_0_512))
res_512_1024 = 0 if len(len_list_512_1024) == 0 else (float(sum(len_list_512_1024)) / len(len_list_512_1024))
print("Average Length of First 512 %s" % res_0_512) 
print("Average Length of Second 512 %s" % res_512_1024) 
print('Found %s texts in First 512.' % len(texts_0_512))
print('Found %s texts in Second 512.' % len(texts_512_1024))

res_1024_1536 = 0 if len(len_list_1024_1536) == 0 else (float(sum(len_list_1024_1536)) / len(len_list_1024_1536))
res_1536_2048 = 0 if len(len_list_1536_2048) == 0 else (float(sum(len_list_1536_2048)) / len(len_list_1536_2048))
print("Average Length of Third 512 %s" % res_1024_1536) 
print("Average Length of Fourth 512 %s" % res_1536_2048) 
print('Found %s texts in Third 512.' % len(texts_1024_1536))
print('Found %s texts in Fourth 512.' % len(texts_1536_2048))

res_2048_2560 = 0 if len(len_list_2048_2560) == 0 else (float(sum(len_list_2048_2560)) / len(len_list_2048_2560))
res_2560_3072 = 0 if len(len_list_2560_3072) == 0 else (float(sum(len_list_2560_3072)) / len(len_list_2560_3072))
print("Average Length of Fifth 512 %s" % res_2048_2560) 
print("Average Length of Sixth 512 %s" % res_2560_3072) 
print('Found %s texts in Fifth 512.' % len(texts_2048_2560))
print('Found %s texts in Sixth 512.' % len(texts_2560_3072))


temp_file = open("labels_sc.txt", "r")

data = temp_file.read()
label_list = data.split("\n")
print(label_list)
label_list = label_list[0:-1]
print(label_list)
label_list = [int(i) for i in label_list]
print('Found %s labels.' % len(set(label_list)))
    

{'name': 'supreme_court', 'site_url': 'http://caselaw.findlaw.com/court/us-supreme-court', 'description': 'Collection of ~8.4k decisions issued by the U.S. Supreme Court between November 1946 and June 2016.'}
[-1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
{'-1': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10, '11': 11, '12': 12, '13': 13, '14': 14}
[8, 1, 8, 2, 8, 8, 8, 9, 7, 8, 1, 2, 1, 1, 8, 4, 8, 8, 12, 3, 3, 7, 3, 12, 1, 8, 8, 8, 8, 8, 8, 8, 1, 9, 5, 9, 9, 9, 11, 8, 8, 8, 4, 8, 8, 8, 8, 1, 3, 9, 3, 7, 1, 2, 9, 9, 7, 8, 8, 1, 10, 7, 8, 8, 9, 8, 7, 9, 9, 12, 7, 2, 8, 1, 11, 11, 1, 7, 7, 12, 1, 9, 8, 10, 12, 7, 8, 2, 8, 9, 9, 1, 8, 9, 1, 7, 12, 10, 10, 10, 8, 3, 7, 9, 8, 9, 1, 8, 8, 2, 7, 2, 9, 9, 11, 8, 8, 12, 12, 7, 8, 12, 4, 9, 3, 1, 12, 1, 1, 8, 8, 3, 8, 8, 8, 8, 9, 1, 8, 8, 10, 1, 8, 2, 8, 8, 7, 3, 8, 2, 4, 4, 9, 8, 10, 12, 12, 1, 1, 9, 1, 1, 1, 8, 2, 2, 8, 1, 1, 2, 2, 1, 2, 8, 1, 9, 9, 8, 8, 4, 2, 9, 9, 8, 3, 4, 3, 1, 8, 8, 2, 1, 9, 7, 8

In [4]:
summarized_data = pd.DataFrame(list(zip(texts_0_512, texts_512_1024,texts_1024_1536,texts_1536_2048,texts_2048_2560,texts_2560_3072)),
               columns =['First_512_text','Second_512_text','Third_512_text','Fourth_512_text','Fifth_512_text','Sixth_512_text'])
summarized_data['label'] = label_list
print(summarized_data)

                                         First_512_text  \
0     [ Halliburton Oil Well Cementing Co. v. Walker...   
1     Rehearing Denied Dec. See . Mr.Claude T. Barne...   
2     Rehearing Denied Dec. See . Appeal from the Di...   
3     Mr. Walter J. Cummings, Jr., of Washington, D....   
4     Mr.A. Devitt Vaneck, of Washington, D.C., for ...   
...                                                 ...   
8414  Opinion reported: Ante, p. DECREE It is ordere...   
8415  In this dispute between Utah and the United St...   
8416  The United States, to the exclusion of defenda...   
8417  Louisiana's exception to the portion of the Sp...   
8418  To resolve a dispute over the ownership of cer...   

                                        Second_512_text  \
0     only had this sound-echo-time method been long...   
1     has no such implied limitation. In common unde...   
2     to apply its conclusion to Champlin. The contr...   
3     size of the reservation; in Congress by statut...

In [5]:

def create_model():
    inps_first_512 = Input(shape = (max_len,), dtype='int64')
    inps_second_512 = Input(shape = (max_len,), dtype='int64')
    inps_third_512 = Input(shape = (max_len,), dtype='int64')
    inps_fourth_512 = Input(shape = (max_len,), dtype='int64')
    inps_fifth_512 = Input(shape = (max_len,), dtype='int64')
    inps_sixth_512 = Input(shape = (max_len,), dtype='int64')

    masks_first_512= Input(shape = (max_len,), dtype='int64')
    masks_second_512= Input(shape = (max_len,), dtype='int64')
    masks_third_512= Input(shape = (max_len,), dtype='int64')
    masks_fourth_512= Input(shape = (max_len,), dtype='int64')
    masks_fifth_512= Input(shape = (max_len,), dtype='int64')
    masks_sixth_512= Input(shape = (max_len,), dtype='int64')

    dbert_layer_first_512 = dbert_model(inps_first_512, attention_mask=masks_first_512)[0][:,0,:]
    dbert_layer_second_512 = dbert_model(inps_second_512, attention_mask=masks_second_512)[0][:,0,:]
    dbert_layer_third_512 = dbert_model(inps_third_512, attention_mask=masks_third_512)[0][:,0,:]
    dbert_layer_fourth_512 = dbert_model(inps_fourth_512, attention_mask=masks_fourth_512)[0][:,0,:]
    dbert_layer_fifth_512 = dbert_model(inps_fifth_512, attention_mask=masks_fifth_512)[0][:,0,:]
    dbert_layer_sixth_512 = dbert_model(inps_sixth_512, attention_mask=masks_sixth_512)[0][:,0,:]

    concat=Concatenate()([dbert_layer_first_512, dbert_layer_second_512,dbert_layer_third_512, dbert_layer_fourth_512,dbert_layer_fifth_512, dbert_layer_sixth_512])
    dense_0 = Dense(512,activation='relu',kernel_regularizer=regularizers.l2(0.001))(concat)
    dropout_0= Dropout(0.5)(dense_0)
    pred = Dense(15, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout_0)
    #pred = Dense(279, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout_0)
    model = tf.keras.Model(inputs=[inps_first_512,masks_first_512,inps_second_512,masks_second_512,inps_third_512,masks_third_512,inps_fourth_512,masks_fourth_512,inps_fifth_512,masks_fifth_512,inps_sixth_512,masks_sixth_512], outputs=pred)
 
    
    print(model.summary())
    return model   

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
total_accuracy=0
total_weighted_f1=0
total_micro_f1=0
total_weighted_precision=0
total_micro_precision=0
total_weighted_recall=0
total_micro_recall=0

for i in range(5):
  gc.collect()
  tf.keras.backend.clear_session()
  dbert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
  dbert_model = TFBertModel.from_pretrained('bert-base-uncased')
  max_len=512
  first_512_sentences=summarized_data['First_512_text']
  second_512_sentences=summarized_data['Second_512_text']
  third_512_sentences=summarized_data['Third_512_text']
  fourth_512_sentences=summarized_data['Fourth_512_text']
  fifth_512_sentences=summarized_data['Fifth_512_text']
  sixth_512_sentences=summarized_data['Sixth_512_text']
  labels=summarized_data['label']
  len(first_512_sentences),len(labels),len(second_512_sentences),len(third_512_sentences),len(fourth_512_sentences),len(fifth_512_sentences),len(sixth_512_sentences)

  model_0=create_model()

  input_ids_first_512=[]
  attention_masks_first_512=[]
  for sent in summarized_data['First_512_text']:
      dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
      input_ids_first_512.append(dbert_inps['input_ids'])
      attention_masks_first_512.append(dbert_inps['attention_mask'])
  input_ids_first_512=np.asarray(input_ids_first_512)
  attention_masks_first_512=np.array(attention_masks_first_512)
  labels=np.array(labels)

  input_ids_second_512=[]
  attention_masks_second_512=[]
  for sent in summarized_data['Second_512_text']:
      dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
      input_ids_second_512.append(dbert_inps['input_ids'])
      attention_masks_second_512.append(dbert_inps['attention_mask'])
  input_ids_second_512=np.asarray(input_ids_second_512)
  attention_masks_second_512=np.array(attention_masks_second_512)

  input_ids_third_512=[]
  attention_masks_third_512=[]
  for sent in summarized_data['Third_512_text']:
      dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
      input_ids_third_512.append(dbert_inps['input_ids'])
      attention_masks_third_512.append(dbert_inps['attention_mask'])
  input_ids_third_512=np.asarray(input_ids_third_512)
  attention_masks_third_512=np.array(attention_masks_third_512)

  input_ids_fourth_512=[]
  attention_masks_fourth_512=[]
  for sent in summarized_data['Fourth_512_text']:
      dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
      input_ids_fourth_512.append(dbert_inps['input_ids'])
      attention_masks_fourth_512.append(dbert_inps['attention_mask'])
  input_ids_fourth_512=np.asarray(input_ids_fourth_512)
  attention_masks_fourth_512=np.array(attention_masks_fourth_512)

  input_ids_fifth_512=[]
  attention_masks_fifth_512=[]
  for sent in summarized_data['Fifth_512_text']:
      dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
      input_ids_fifth_512.append(dbert_inps['input_ids'])
      attention_masks_fifth_512.append(dbert_inps['attention_mask'])
  input_ids_fifth_512=np.asarray(input_ids_fifth_512)
  attention_masks_fifth_512=np.array(attention_masks_fifth_512)

  input_ids_sixth_512=[]
  attention_masks_sixth_512=[]
  for sent in summarized_data['Sixth_512_text']:
      dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
      input_ids_sixth_512.append(dbert_inps['input_ids'])
      attention_masks_sixth_512.append(dbert_inps['attention_mask'])
  input_ids_sixth_512=np.asarray(input_ids_sixth_512)
  attention_masks_sixth_512=np.array(attention_masks_sixth_512)

  train_inp_first_512,val_inp_first_512,train_label,val_label,train_mask_first_512,val_mask_first_512,train_inp_second_512,val_inp_second_512,train_mask_second_512,val_mask_second_512,train_inp_third_512,val_inp_third_512,train_mask_third_512,val_mask_third_512,train_inp_fourth_512,val_inp_fourth_512,train_mask_fourth_512,val_mask_fourth_512,train_inp_fifth_512,val_inp_fifth_512,train_mask_fifth_512,val_mask_fifth_512,train_inp_sixth_512,val_inp_sixth_512,train_mask_sixth_512,val_mask_sixth_512=train_test_split(input_ids_first_512,labels,attention_masks_first_512,input_ids_second_512,attention_masks_second_512,input_ids_third_512,attention_masks_third_512,input_ids_fourth_512,attention_masks_fourth_512,input_ids_fifth_512,attention_masks_fifth_512,input_ids_sixth_512,attention_masks_sixth_512,test_size=0.1,random_state=42)

  log_dir='dbert_model'

  model_save_path='Concat-512/bert-concat-512-'+str(i)+'-15labels.h5'

  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

  optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
  callbacks= [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,monitor='val_accuracy',mode='max',save_best_only=True,save_weights_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]
  model_0.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])

  gpu_info = !nvidia-smi
  gpu_info = '\n'.join(gpu_info)
  if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
  else:
    print(gpu_info)
  
  history=model_0.fit([train_inp_first_512,train_mask_first_512,train_inp_second_512,train_mask_second_512,train_inp_third_512,train_mask_third_512,train_inp_fourth_512,train_mask_fourth_512,train_inp_fifth_512,train_mask_fifth_512,train_inp_sixth_512,train_mask_sixth_512],train_label,batch_size=8,epochs=5,validation_data=([val_inp_first_512,val_mask_first_512,val_inp_second_512,val_mask_second_512,val_inp_third_512,val_mask_third_512,val_inp_fourth_512,val_mask_fourth_512,val_inp_fifth_512,val_mask_fifth_512,val_inp_sixth_512,val_mask_sixth_512],val_label),callbacks=callbacks)
  pred_labels=[]

  model_saved= create_model()
  model_saved.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])
  model_saved.load_weights('Concat-512/bert-concat-512-'+str(i)+'-15labels.h5')

  for i in range(0,len(val_inp_first_512)):
    pred=model_saved.predict([val_inp_first_512[i].reshape(1,512),val_mask_first_512[i].reshape(1,512),val_inp_second_512[i].reshape(1,512),val_mask_second_512[i].reshape(1,512),val_inp_third_512[i].reshape(1,512),val_mask_third_512[i].reshape(1,512),val_inp_fourth_512[i].reshape(1,512),val_mask_fourth_512[i].reshape(1,512),val_inp_fifth_512[i].reshape(1,512),val_mask_fifth_512[i].reshape(1,512),val_inp_sixth_512[i].reshape(1,512),val_mask_sixth_512[i].reshape(1,512)])
    pred_label = pred.argmax(axis=1)
    pred_labels.append(pred_label)
  accuracy=accuracy_score(val_label, pred_labels)
  print("Accuracy: "+str(accuracy))
  total_accuracy=total_accuracy+accuracy
  
  weighted_f1=f1_score(val_label,pred_labels, average='weighted')
  print("Weighted F1: "+ str(weighted_f1))
  total_weighted_f1=total_weighted_f1+weighted_f1
  micro_f1=f1_score(val_label,pred_labels, average='micro')
  print("Micro F1: "+ str(micro_f1))
  total_micro_f1=total_micro_f1+micro_f1

  weighted_precision=precision_score(val_label, pred_labels, average='weighted')
  print("Weighted Precision: " + str(weighted_precision))
  total_weighted_precision=total_weighted_precision+weighted_precision
  micro_precision=precision_score(val_label, pred_labels, average='micro')
  print("Micro Precision: " + str(micro_precision))
  total_micro_precision=total_micro_precision+micro_precision

  weighted_recall=recall_score(val_label, pred_labels, average='weighted')
  print("Weighted Recall: " + str(weighted_recall))
  total_weighted_recall=total_weighted_recall+weighted_recall
  micro_recall=recall_score(val_label, pred_labels, average='micro')
  print("Micro Recall: " + str(micro_recall))
  total_micro_recall=total_micro_recall+micro_recall


print("Average Accuracy: "+str(total_accuracy/5))
print("Average Weighted F1: "+str(total_weighted_f1/5))
print("Average Micro F1: "+str(total_micro_f1/5))
print("Average Weighted Precision: "+str(total_weighted_precision/5))
print("Average Micro Precision: "+str(total_micro_precision/5))
print("Average Weighted Recall: "+str(total_weighted_recall/5))
print("Average Micro Recall: "+str(total_micro_recall/5))

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 input_7 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 input_8 (InputLayer)           [(None, 512)]        0           []                               
                                                                                              

/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Sun Jun  5 16:39:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:4B:00.0 Off |                    0 |
| N/A   26C    P0    75W / 500W |  79808MiB / 81920MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


...

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 input_7 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 input_8 (InputLayer)           [(None, 512)]        0           []                               
                                                                                              

/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Sun Jun  5 19:35:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:4B:00.0 Off |                    0 |
| N/A   26C    P0    78W / 500W |  79808MiB / 81920MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


693/948 [====================>.........] - ETA: 2:31 - loss: 2.5696 - accuracy: 0.5675

In [12]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score


for i in range(1):
  gc.collect()
  tf.keras.backend.clear_session()
  dbert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
  dbert_model = TFBertModel.from_pretrained('bert-base-uncased')
  max_len=512
  first_512_sentences=summarized_data['First_512_text']
  second_512_sentences=summarized_data['Second_512_text']
  third_512_sentences=summarized_data['Third_512_text']
  fourth_512_sentences=summarized_data['Fourth_512_text']
  fifth_512_sentences=summarized_data['Fifth_512_text']
  sixth_512_sentences=summarized_data['Sixth_512_text']
  labels=summarized_data['label']
  len(first_512_sentences),len(labels),len(second_512_sentences),len(third_512_sentences),len(fourth_512_sentences),len(fifth_512_sentences),len(sixth_512_sentences)

  model_0=create_model()

  input_ids_first_512=[]
  attention_masks_first_512=[]
  for sent in summarized_data['First_512_text']:
      dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
      input_ids_first_512.append(dbert_inps['input_ids'])
      attention_masks_first_512.append(dbert_inps['attention_mask'])
  input_ids_first_512=np.asarray(input_ids_first_512)
  attention_masks_first_512=np.array(attention_masks_first_512)
  labels=np.array(labels)

  input_ids_second_512=[]
  attention_masks_second_512=[]
  for sent in summarized_data['Second_512_text']:
      dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
      input_ids_second_512.append(dbert_inps['input_ids'])
      attention_masks_second_512.append(dbert_inps['attention_mask'])
  input_ids_second_512=np.asarray(input_ids_second_512)
  attention_masks_second_512=np.array(attention_masks_second_512)

  input_ids_third_512=[]
  attention_masks_third_512=[]
  for sent in summarized_data['Third_512_text']:
      dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
      input_ids_third_512.append(dbert_inps['input_ids'])
      attention_masks_third_512.append(dbert_inps['attention_mask'])
  input_ids_third_512=np.asarray(input_ids_third_512)
  attention_masks_third_512=np.array(attention_masks_third_512)

  input_ids_fourth_512=[]
  attention_masks_fourth_512=[]
  for sent in summarized_data['Fourth_512_text']:
      dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
      input_ids_fourth_512.append(dbert_inps['input_ids'])
      attention_masks_fourth_512.append(dbert_inps['attention_mask'])
  input_ids_fourth_512=np.asarray(input_ids_fourth_512)
  attention_masks_fourth_512=np.array(attention_masks_fourth_512)

  input_ids_fifth_512=[]
  attention_masks_fifth_512=[]
  for sent in summarized_data['Fifth_512_text']:
      dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
      input_ids_fifth_512.append(dbert_inps['input_ids'])
      attention_masks_fifth_512.append(dbert_inps['attention_mask'])
  input_ids_fifth_512=np.asarray(input_ids_fifth_512)
  attention_masks_fifth_512=np.array(attention_masks_fifth_512)

  input_ids_sixth_512=[]
  attention_masks_sixth_512=[]
  for sent in summarized_data['Sixth_512_text']:
      dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
      input_ids_sixth_512.append(dbert_inps['input_ids'])
      attention_masks_sixth_512.append(dbert_inps['attention_mask'])
  input_ids_sixth_512=np.asarray(input_ids_sixth_512)
  attention_masks_sixth_512=np.array(attention_masks_sixth_512)

  train_inp_first_512,val_inp_first_512,train_label,val_label,train_mask_first_512,val_mask_first_512,train_inp_second_512,val_inp_second_512,train_mask_second_512,val_mask_second_512,train_inp_third_512,val_inp_third_512,train_mask_third_512,val_mask_third_512,train_inp_fourth_512,val_inp_fourth_512,train_mask_fourth_512,val_mask_fourth_512,train_inp_fifth_512,val_inp_fifth_512,train_mask_fifth_512,val_mask_fifth_512,train_inp_sixth_512,val_inp_sixth_512,train_mask_sixth_512,val_mask_sixth_512=train_test_split(input_ids_first_512,labels,attention_masks_first_512,input_ids_second_512,attention_masks_second_512,input_ids_third_512,attention_masks_third_512,input_ids_fourth_512,attention_masks_fourth_512,input_ids_fifth_512,attention_masks_fifth_512,input_ids_sixth_512,attention_masks_sixth_512,test_size=0.1,random_state=42)

  log_dir='dbert_model'

  model_save_path='Concat-512/bert-concat-512-4-15labels.h5'

  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

  optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
  callbacks= [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,monitor='val_accuracy',mode='max',save_best_only=True,save_weights_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]
  model_0.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])

  gpu_info = !nvidia-smi
  gpu_info = '\n'.join(gpu_info)
  if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
  else:
    print(gpu_info)
  
  history=model_0.fit([train_inp_first_512,train_mask_first_512,train_inp_second_512,train_mask_second_512,train_inp_third_512,train_mask_third_512,train_inp_fourth_512,train_mask_fourth_512,train_inp_fifth_512,train_mask_fifth_512,train_inp_sixth_512,train_mask_sixth_512],train_label,batch_size=8,epochs=5,validation_data=([val_inp_first_512,val_mask_first_512,val_inp_second_512,val_mask_second_512,val_inp_third_512,val_mask_third_512,val_inp_fourth_512,val_mask_fourth_512,val_inp_fifth_512,val_mask_fifth_512,val_inp_sixth_512,val_mask_sixth_512],val_label),callbacks=callbacks)
  pred_labels=[]

  model_saved= create_model()
  model_saved.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])
  model_saved.load_weights('Concat-512/bert-concat-512-4-15labels.h5')

  for i in range(0,len(val_inp_first_512)):
    pred=model_saved.predict([val_inp_first_512[i].reshape(1,512),val_mask_first_512[i].reshape(1,512),val_inp_second_512[i].reshape(1,512),val_mask_second_512[i].reshape(1,512),val_inp_third_512[i].reshape(1,512),val_mask_third_512[i].reshape(1,512),val_inp_fourth_512[i].reshape(1,512),val_mask_fourth_512[i].reshape(1,512),val_inp_fifth_512[i].reshape(1,512),val_mask_fifth_512[i].reshape(1,512),val_inp_sixth_512[i].reshape(1,512),val_mask_sixth_512[i].reshape(1,512)])
    pred_label = pred.argmax(axis=1)
    pred_labels.append(pred_label)
  accuracy=accuracy_score(val_label, pred_labels)
  print("Accuracy: "+str(accuracy))
  
  weighted_f1=f1_score(val_label,pred_labels, average='weighted')
  print("Weighted F1: "+ str(weighted_f1))
  micro_f1=f1_score(val_label,pred_labels, average='micro')
  print("Micro F1: "+ str(micro_f1))
  
  weighted_precision=precision_score(val_label, pred_labels, average='weighted')
  print("Weighted Precision: " + str(weighted_precision))
  micro_precision=precision_score(val_label, pred_labels, average='micro')
  print("Micro Precision: " + str(micro_precision))
  
  weighted_recall=recall_score(val_label, pred_labels, average='weighted')
  print("Weighted Recall: " + str(weighted_recall))
  micro_recall=recall_score(val_label, pred_labels, average='micro')
  print("Micro Recall: " + str(micro_recall))

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 input_7 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 input_8 (InputLayer)           [(None, 512)]        0           []                               
                                                                                              

/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Sun Jun  5 22:38:14 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:4B:00.0 Off |                    0 |
| N/A   26C    P0    78W / 500W |  79808MiB / 81920MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


948/948 [==============================] - 690s 627ms/step - loss: 2.4793 - accuracy: 0.5967 - val_loss: 2.0637 - val_accuracy: 0.7031
Epoch 2/5
948/948 [==============================] - 581s 613ms/step - loss: 1.8887 - accuracy: 0.7593 - val_loss: 1.8604 - val_accuracy: 0.7672
Epoch 3/5
948/948 [==============================] - 581s 613ms/step - loss: 1.6171 - accuracy: 0.8193 - val_loss: 1.8782 - val_accuracy: 0.7447
Epoch 4/5
948/948 [==============================] - 581s 613ms/step - loss: 1.4125 - accuracy: 0.8737 - val_loss: 1.8290 - val_accuracy: 0.7577
Epoch 5/5
948/948 [==============================] - 581s 613ms/step - loss: 1.2377 - accuracy: 0.9150 - val_loss: 1.8269 - val_accuracy: 0.7648
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_13 (InputLayer)          [(None, 512)]        0           []   

/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
total_accuracy=0
total_weighted_f1=0
total_micro_f1=0
total_weighted_precision=0
total_micro_precision=0
total_weighted_recall=0
total_micro_recall=0


for i in range(5):
    pred_labels=[]
    model_saved= create_model()
    model_saved.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])
    model_saved.load_weights('Concat-512/bert-concat-512-'+str(i)+'-15labels.h5')

    for i in range(0,len(val_inp_first_512)):
        pred=model_saved.predict([val_inp_first_512[i].reshape(1,512),val_mask_first_512[i].reshape(1,512),val_inp_second_512[i].reshape(1,512),val_mask_second_512[i].reshape(1,512),val_inp_third_512[i].reshape(1,512),val_mask_third_512[i].reshape(1,512),val_inp_fourth_512[i].reshape(1,512),val_mask_fourth_512[i].reshape(1,512),val_inp_fifth_512[i].reshape(1,512),val_mask_fifth_512[i].reshape(1,512),val_inp_sixth_512[i].reshape(1,512),val_mask_sixth_512[i].reshape(1,512)])
        pred_label = pred.argmax(axis=1)
        pred_labels.append(pred_label)
    accuracy=accuracy_score(val_label, pred_labels)
    print("Accuracy: "+str(accuracy))
    total_accuracy=total_accuracy+accuracy

    weighted_f1=f1_score(val_label,pred_labels, average='weighted')
    print("Weighted F1: "+ str(weighted_f1))
    total_weighted_f1=total_weighted_f1+weighted_f1
    micro_f1=f1_score(val_label,pred_labels, average='micro')
    print("Micro F1: "+ str(micro_f1))
    total_micro_f1=total_micro_f1+micro_f1

    weighted_precision=precision_score(val_label, pred_labels, average='weighted')
    print("Weighted Precision: " + str(weighted_precision))
    total_weighted_precision=total_weighted_precision+weighted_precision
    micro_precision=precision_score(val_label, pred_labels, average='micro')
    print("Micro Precision: " + str(micro_precision))
    total_micro_precision=total_micro_precision+micro_precision

    weighted_recall=recall_score(val_label, pred_labels, average='weighted')
    print("Weighted Recall: " + str(weighted_recall))
    total_weighted_recall=total_weighted_recall+weighted_recall
    micro_recall=recall_score(val_label, pred_labels, average='micro')
    print("Micro Recall: " + str(micro_recall))
    total_micro_recall=total_micro_recall+micro_recall
    
print("Average Accuracy: "+str(total_accuracy/5))
print("Average Weighted F1: "+str(total_weighted_f1/5))
print("Average Micro F1: "+str(total_micro_f1/5))
print("Average Weighted Precision: "+str(total_weighted_precision/5))
print("Average Micro Precision: "+str(total_micro_precision/5))
print("Average Weighted Recall: "+str(total_weighted_recall/5))
print("Average Micro Recall: "+str(total_micro_recall/5))

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_37 (InputLayer)          [(None, 512)]        0           []                               
                                                                                                  
 input_43 (InputLayer)          [(None, 512)]        0           []                               
                                                                                                  
 input_38 (InputLayer)          [(None, 512)]        0           []                               
                                                                                                  
 input_44 (InputLayer)          [(None, 512)]        0           []                               
                                                                                            

/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_49 (InputLayer)          [(None, 512)]        0           []                               
                                                                                                  
 input_55 (InputLayer)          [(None, 512)]        0           []                               
                                                                                                  
 input_50 (InputLayer)          [(None, 512)]        0           []                               
                                                                                                  
 input_56 (InputLayer)          [(None, 512)]        0           []                               
                                                                                            

/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_61 (InputLayer)          [(None, 512)]        0           []                               
                                                                                                  
 input_67 (InputLayer)          [(None, 512)]        0           []                               
                                                                                                  
 input_62 (InputLayer)          [(None, 512)]        0           []                               
                                                                                                  
 input_68 (InputLayer)          [(None, 512)]        0           []                               
                                                                                            

/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_73 (InputLayer)          [(None, 512)]        0           []                               
                                                                                                  
 input_79 (InputLayer)          [(None, 512)]        0           []                               
                                                                                                  
 input_74 (InputLayer)          [(None, 512)]        0           []                               
                                                                                                  
 input_80 (InputLayer)          [(None, 512)]        0           []                               
                                                                                            

/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_85 (InputLayer)          [(None, 512)]        0           []                               
                                                                                                  
 input_91 (InputLayer)          [(None, 512)]        0           []                               
                                                                                                  
 input_86 (InputLayer)          [(None, 512)]        0           []                               
                                                                                                  
 input_92 (InputLayer)          [(None, 512)]        0           []                               
                                                                                            